In [1]:
import os
SampleDataDir = "C:/Users/Rishabh/Documents/DICOMIZER/SampleData/example_input_images"
ceT1 = os.path.join(SampleDataDir,'ceT1')
ceT1_T2 = os.path.join(SampleDataDir,'ceT1+T2')
T2 = os.path.join(SampleDataDir,'T2')

In [2]:
from nii2dcm.run import run_nii2dcm

In [ ]:
nii_file = os.path.join(ceT1, os.listdir(ceT1)[0])
output_dcm_path = 'dicom-output-directory'
dicom_type = 'MR'
run_nii2dcm(nii_file,output_dcm_path, dicom_type )

In [3]:
import shutil
# shutil.make_archive(output_archive, 'zip', directory_path)

In [6]:
import pydicom
import shutil
import os

def fun(niftypath, outputDirPath, outputzipPath ):
    shutil.rmtree(outputDirPath)
    os.makedirs(outputDirPath, exist_ok=True)
    run_nii2dcm(niftypath,outputDirPath, 'MR')
    for dcm in os.listdir(outputDirPath):
        os.rename(os.path.join(outputDirPath,dcm),os.path.join(outputDirPath,dcm)+'.dcm')
    new_study_id = "12345"
    # Iterate through each file in the directory
    for filename in os.listdir(outputDirPath):
        # Build the full file path
        file_path = os.path.join(outputDirPath, filename)
        # Check if the file is a valid DICOM file
        if filename.endswith(".dcm"):  # Ensures only .dcm files are processed
            try:
                # Load the DICOM file
                ds = pydicom.dcmread(file_path)
                # Check if the StudyID tag exists
                if 'StudyID' in ds:
                    print(f"Original StudyID in {filename}: {ds.StudyID}")
                else:
                    print(f"StudyID missing in {filename}. Adding it now.")
                # Set or update the StudyID
                ds.StudyID = new_study_id
                # Save the file in place (overwrite the original file)
                ds.save_as(file_path)
                print(f"Updated StudyID for {filename} and saved in place.")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")
    shutil.make_archive(outputzipPath, 'zip', outputDirPath)

In [8]:
niftypath = os.path.join(T2, os.listdir(T2)[0])
outputDirPath = 'dicom-output-directory'
outputzipPath = os.path.join('output', 'compressed' )
fun(niftypath, outputDirPath, outputzipPath )

nii2dcm: writing DICOM files ...
nii2dcm: DICOM files written to: C:\Users\Rishabh\Documents\DICOMIZER\dicom-output-directory
StudyID missing in IM_0001.dcm. Adding it now.
Updated StudyID for IM_0001.dcm and saved in place.
StudyID missing in IM_0002.dcm. Adding it now.
Updated StudyID for IM_0002.dcm and saved in place.
StudyID missing in IM_0003.dcm. Adding it now.
Updated StudyID for IM_0003.dcm and saved in place.
StudyID missing in IM_0004.dcm. Adding it now.
Updated StudyID for IM_0004.dcm and saved in place.
StudyID missing in IM_0005.dcm. Adding it now.
Updated StudyID for IM_0005.dcm and saved in place.
StudyID missing in IM_0006.dcm. Adding it now.
Updated StudyID for IM_0006.dcm and saved in place.
StudyID missing in IM_0007.dcm. Adding it now.
Updated StudyID for IM_0007.dcm and saved in place.
StudyID missing in IM_0008.dcm. Adding it now.
Updated StudyID for IM_0008.dcm and saved in place.
StudyID missing in IM_0009.dcm. Adding it now.
Updated StudyID for IM_0009.dcm and

In [11]:
import nibabel as nib
import numpy as np

segmentationfile = 'SampleData/vs_gk.nii.gz'
img = nib.load(segmentationfile)

# Get the data as a NumPy array
data = img.get_fdata()
data = np.array(data)
print(data.shape)
print(np.unique(data))

(416, 488, 81)
[0. 1.]


In [12]:
import highdicom as hd
from pydicom.sr.codedict import codes

segmentdescriptions = []
for i in range(len(np.unique(data))-1):
    description = hd.seg.SegmentDescription(
    segment_number=i+1,
    segment_label=f'Tumor_{i}',
    segmented_property_category=codes.SCT.Blood,
    segmented_property_type=codes.SCT.Blood,
    algorithm_type=hd.seg.SegmentAlgorithmTypeValues.MANUAL,
    )
    segmentdescriptions.append(description)

# description = hd.seg.SegmentDescription(
#     segment_number=1,
#     segment_label='Tumor',
#     segmented_property_category=codes.SCT.Blood,
#     segmented_property_type=codes.SCT.Blood,
#     algorithm_type=hd.seg.SegmentAlgorithmTypeValues.MANUAL,
# )

In [13]:
import os
import pydicom
dicom_output_directory = 'dicom-output-directory'
files = os.listdir(dicom_output_directory)
files = sorted(files)
maskDict = {}
maskDict['Tumor'] = []
source_images = []
i = 0
for file in files:
    file_path = os.path.join(dicom_output_directory, file)
    source_images.append(pydicom.dcmread(file_path))
    slice1 = data[:,:,i]
    maskDict['Tumor'].append(np.array(slice1))
    i = i+ 1
    

In [14]:
mask = []
for key,value in maskDict.items():
    mask.append(np.stack(value, axis=0))

In [15]:
mask = np.stack(mask, axis=3)
seg_obj = hd.seg.Segmentation(
    source_images=source_images,
    pixel_array=mask,
    segmentation_type=hd.seg.SegmentationTypeValues.BINARY,
    segment_descriptions=segmentdescriptions,
    series_instance_uid=hd.UID(),
    series_number=1,
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer='Radpretation ai',
    manufacturer_model_name='Brain structure Segmentation Algorithm',
    software_versions='0.0.1',
    device_serial_number='1234567890'
)

In [16]:
seg_obj.save_as('dicom-output-directory/AI.dcm')